<a href="https://colab.research.google.com/github/bassamadnan/deep-learning-dog-breed-classification/blob/main/dog_breed_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐕‍🦺 End-to-end Multi-class Dog Breed Classification
This notebook  builds an end-to-end multi-class image classifier using TensorFlow 2.0 and TensorFlow Hub.

## 1. Problem
> Identifying the breed of a dog given an image of a dog.

## 2. Data
The data we are using is from Kaggle's dog breed identification competition.
https://www.kaggle.com/competitions/dog-breed-identification/overview

## 3. Evaluation
The evaluation is a file with prediction probabilities for each dog breed of each test image.
https://www.kaggle.com/competitions/dog-breed-identification/overview/evaluation

## 4. Features
Some information about the data:
* We're dealing with images (unstructured data) so it's probably best we use Deep Learning/ Transfer Learning
* There are 120 breeds of dogs (this means there are 120 different classes).
* There are 10,000+ images in the training set (labelled)
* There are around 10,000 images in the test set (unlabelled and to be predicted)

In [10]:
# Unzip the data 
# !unzip "/content/drive/MyDrive/Dog Vision/dog-breed-identification.zip" -d "drive/MyDrive/Dog Vision/"

## Get our workspace ready

In [1]:
# Import TensorFlow into Colab
import tensorflow as tf
import tensorflow_hub as hub
print("TF version: ", tf.__version__) 
print("TF Hub: ", hub.__version__)

# Check for GPU availability
print("GPU", "available (!!)" if tf.config.list_physical_devices("GPU") else ":(")

TF version:  2.8.2
TF Hub:  0.12.0
GPU available (!!)
